## 0. 라이브러리 호출:

1. **os**: 이 라이브러리는 운영 체제에 의존하는 기능을 사용할 수 있게 해 주는 파이썬 모듈입니다. 파이썬 스크립트가 실행되는 운영 체제와 상호 작용할 수 있도록 합니다.

2. **logging**: 애플리케이션 및 라이브러리를 위한 유연한 이벤트 로깅 시스템을 구현하는 함수와 클래스를 정의하는 모듈입니다.

3. **gtts (Google Text-to-Speech)**: 이 라이브러리는 텍스트를 음성으로 변환하는 데 사용됩니다. 구글 텍스트 음성 API를 사용합니다.

4. **pdfplumber**: PDF에서 텍스트, 이미지 및 테이블을 추출하기 위한 라이브러리입니다.

5. **langchain**: 이 라이브러리는 자연어 처리(NLP) 작업을 위한 사용자 정의 라이브러리인 것 같습니다. 벡터 저장소, 임베딩, 텍스트 분할, 검색 질의 응답 및 채팅 모델과 관련된 기능을 제공합니다. 이 라이브러리는 일반적인 파이썬 라이브러리나 NLTK나 spaCy와 같은 인기 있는 NLP 라이브러리로는 제공되지 않는 고급 NLP 작업을 수행하는 데 사용될 것 같습니다.

6. **speech_recognition**: 이 라이브러리는 다양한 오디오 소스에서 음성을 인식하는 간단한 인터페이스를 제공합니다. 여러 음성 인식 엔진과 API를 지원합니다.

7. **tkinter**: 이것은 Tk GUI 툴킷의 표준 파이썬 인터페이스입니다. 파이썬에서 그래픽 사용자 인터페이스(GUI)를 만드는 데 사용됩니다.

8. **playsound**: 오디오 파일을 재생하기 위한 순수 파이썬, 크로스 플랫폼 라이브러리입니다.

#### 0-1. 코드 스니펫:

1. **모듈 가져오기**: 여러분은 os, logging, gtts, pdfplumber, langchain, speech_recognition, tkinter 및 playsound와 같은 여러 모듈과 패키지를 스크립트에 필요한 것으로 가져왔습니다.

2. **모듈로부터의 클래스/함수 가져오기**: langchain에서 특정 클래스나 함수들을 가져왔습니다. 예를 들어, `gtts`에서 `gTTS`, `langchain.vectorstores`에서 `Chroma`와 `OpenAIEmbeddings`, 그리고 `langchain`에서 `RecursiveCharacterTextSplitter`, `RetrievalQA`, `ChatOpenAI`를 가져왔습니다.

3. **Document 객체**: `langchain.docstore.document`에서 `Document` 객체에 대한 참조가 있지만, 제공된 코드 스니펫에서는 인스턴스화되거나 사용되지 않았습니다.

4. **LLMChain**: `langchain.chains`에서 또 다른 클래스를 가져왔으나, 이 코드 스니펫에서는 사용되지 않았습니다.

5. **GUI 구성 요소**: GUI를 구축하기 위해 `tkinter`와 `messagebox`를 가져왔습니다. 그러나 이 코드 스니펫에서는 GUI를 만드는 코드가 제공되지 않았습니다.

6. **Playsound**: 오디오 파일을 재생하기 위해 `playsound`를 가져왔지만, 제공된 코드 스니펫에서는 사용되지 않았습니다.

전반적으로, 텍스트 처리, 음성 인식, 텍스트 음성 변환 및 이러한 기능과 상호 작용하는 GUI를 구축하는 프로젝트

In [1]:
import os
import logging
from gtts import gTTS
import pdfplumber
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import speech_recognition as sr
import tkinter as tk
from tkinter import messagebox
from playsound import playsound

## 1. OpenAI API를 사용하기 위해 환경 변수에 API 키를 설정

In [2]:
os.environ["OPENAI_API_KEY"] = os.environ.get('OPENAI_API_KEY')

## 2. 로그 저장하기 위한 설정

In [3]:
# 로그 파일의 이름과 로그 레벨, 포맷을 설정하여 기본 로그 설정을 수행합니다.
# 로그는 'query_log.txt' 파일에 저장되며, 파일 모드는 'a'로 기존 로그를 추가합니다.
logging.basicConfig(filename='query_log.txt',
                    level=logging.INFO,  # 로그 레벨을 INFO로 설정하여 info 이상의 메시지만 기록합니다.
                    format='%(asctime)s - %(message)s',  # 로그 포맷은 시간 정보와 메시지만 포함합니다.
                    filemode='a')

# 쿼리와 응답을 로깅하는 함수를 정의합니다.
def log_query_and_response(query, response):
    # logging 모듈의 info 함수를 사용하여 쿼리와 응답을 로그에 기록합니다.
    logging.info(f"Query: {query}")  # 쿼리를 로그에 기록합니다.
    logging.info(f"Response: {response}")  # 응답을 로그에 기록합니다.

## 3. pdfplumber를 사용한 PDF에서 테이블 추출 함수.

In [4]:
def extract_tables_from_pdfplumber(pdf_path):
    tables_content = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                tables_content.append(table)
    return tables_content

## 4. 지정 디렉토리 내의 PDF 파일을 처리 후 문서변환 함수.

In [5]:
def process_pdfs(directory):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            tables_content = extract_tables_from_pdfplumber(file_path)
            for table_data in tables_content:
                table_text = '\n'.join(['\t'.join([str(cell) if cell is not None else '' for cell in row]) for row in table_data])
                documents.append(Document(page_content=table_text, metadata={"source": filename}))
    return documents

## 5. 벡터 스토어 기능
#### 5-1. 벡터 스토어를 저장할 디렉토리를 지정합니다.
이 코드는 벡터 스토어가 저장될 디렉토리의 경로를 지정합니다. 여기서는 'car_info_sql'이라는 디렉토리를 사용합니다.

#### 5-2. 벡터 스토어를 생성하거나 로드
디렉토리가 존재하지 않으면, PDF 파일을 처리하여 문서를 생성합니다. `process_pdfs` 함수는 지정된 디렉토리('./pdfs/')에 있는 PDF 파일들을 처리합니다.

생성된 문서를 청크(작은 단위)로 나누기 위해 `RecursiveCharacterTextSplitter`를 사용합니다. 여기서는 각 청크의 크기를 1000자로 하고, 청크 간에 200자의 중복을 둡니다.

나눠진 텍스트 청크들을 임베딩하여 벡터 스토어를 생성합니다. `OpenAIEmbeddings`를 사용하여 텍스트를 벡터로 변환하고, `Chroma`를 사용하여 벡터 스토어를 생성하고 지정된 디렉토리에 저장합니다. `vectordb.persist()`는 생성된 벡터 스토어를 디스크에 저장하는 역할을 합니다.

디렉토리가 이미 존재하면, 저장된 벡터 스토어를 로드합니다. `Chroma`를 사용하여 지정된 디렉토리에서 벡터 스토어를 불러옵니다.

In [6]:
# 벡터 스토어를 저장할 디렉토리를 지정합니다.
persist_directory = 'car_info_sql'

# 벡터 스토어를 생성하거나 로드
if not os.path.exists(persist_directory):
    # PDF 파일을 처리하여 문서 생성
    documents = process_pdfs('./pdfs/')

    # 문서를 청크로 나눔
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(documents)

    # 임베딩과 벡터 스토어 생성
    embedding = OpenAIEmbeddings()
    vectordb = Chroma.from_documents(documents=texts,
                                     embedding=embedding,
                                     persist_directory=persist_directory)
    vectordb.persist()  # 한 번만 호출
else:
    # 벡터 스토어 로드
    vectordb = Chroma(persist_directory=persist_directory,
                      embedding_function=OpenAIEmbeddings())

/opt/anaconda3/envs/CHAT_BOT_SERVICE/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## 6. 벡터 스토어를 사용, RetrievalQA 체인을 초기화.

In [7]:
# RetrievalQA 체인 생성
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

## 7. 사용자-시스템 간 채팅 프롬프트 템플릿 설정.

In [8]:
# 프롬프트 템플릿 초기화
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "안녕하세요! 차량 정보를 정확히 제공해드릴 수 있는 시스템입니다."
               "현재 데이터베이스에 '기아 K3'와 '폭스바겐 골프 7세대'의 정보가 있습니다."
               "질문하실 때 특정 차량을 정확하게 구분해주세요."
               "데이터는 영어와 한국어로 구성되어 있으며, 답변은 한국어로 드립니다."
               "길이 단위는 기본적으로 미터(m)로 사용합니다."
               "데이터에 마일(mile)이 있을 경우 미터로 환산해 드릴게요."
               "차량 정보를 혼동하지 않도록 주의할게요."
               "상세한 설명을 요청하셔도 답변은 250자 이내로 간결하게 드립니다."
               "사람과 대화하듯 편하게 구어체로 답변해 주되 존댓말로 답변해 주세요."
               ),
    ("user", "{user_input}"),
])

## 8. 모델 체이닝
#### 8-1. GPT-4o 모델로 채팅 모델 초기화
(최대한 정확하게 토큰수는 최대로)이 코드는 `ChatOpenAI` 클래스를 사용하여 GPT-4o 모델을 초기화합니다. `temperature`는 0으로 설정하여 출력을 결정론적으로 만들고, `max_tokens`는 2048로 설정하여 가능한 최대 토큰 수를 사용합니다.

#### 8-2. 언어 모델과 프롬프트 템플릿을 포함한 체인 초기화
이 코드는 `LLMChain` 클래스를 사용하여 언어 모델과 프롬프트 템플릿을 포함한 체인을 초기화합니다. `llm`은 위에서 초기화한 GPT-4o 모델이고, `prompt`는 `chat_prompt`를 사용하며, 출력은 'result' 키에 저장됩니다.

#### 8-3. RetrievalQA 체인 생성
이 코드는 `RetrievalQA` 클래스를 사용하여 RetrievalQA 체인을 생성합니다. `llm`은 위에서 초기화한 GPT-4o 모델이고, `chain_type`은 "stuff"로 설정됩니다. `retriever`는 검색을 수행할 리트리버 객체이며, `return_source_documents`를 `True`로 설정하여 반환된 답변과 함께 관련 소스 문서도 반환되도록 합니다.

In [9]:
# GPT-4o 모델로 채팅 모델 초기화 (최대한 정확하게 토큰수는 최대로)
llm = ChatOpenAI(model_name="gpt-4o",
                 temperature=0,
                 max_tokens=2048)

# 언어 모델과 프롬프트 템플릿을 포함한 체인 초기화
chain = LLMChain(llm=llm, prompt=chat_prompt, output_key='result')

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True  # 문서 정보 반환 옵션 추가
)

/opt/anaconda3/envs/CHAT_BOT_SERVICE/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## 9. 음성 관련 모듈
#### 9-1. 텍스트 변환 함수
이 함수는 입력된 텍스트에서 특정 단위를 한글 표현으로 변환합니다. `replace` 메서드를 사용하여 "L"을 "리터"로, "kg.m"을 "킬로그램퍼미터"로, "cc"를 "씨씨"로, "kg"을 "킬로그램"으로, "km"을 "킬로미터"로, "rpm"을 "알피엠"으로 변환합니다.

#### 9-2. 음성 출력 함수
이 함수는 텍스트를 음성으로 변환하고 재생합니다.

1. `cleaned_text = text.replace('*', '').replace('**', '')`: 텍스트에서 '*'와 '**'를 제거합니다.
2. `cleaned_text = convert_text(cleaned_text)`: `convert_text` 함수를 호출하여 텍스트 내의 특정 단위를 한글 표현으로 변환합니다.
3. `tts = gTTS(text=cleaned_text, lang=lang)`: `gTTS`를 사용하여 변환된 텍스트를 음성으로 변환합니다. 언어는 기본적으로 한국어('ko')로 설정됩니다.
4. `tts.save("output.mp3")`: 생성된 음성 파일을 "output.mp3"로 저장합니다.
5. `playsound("output.mp3")`: 저장된 음성 파일을 재생합니다.

In [10]:
# 텍스트 변환 함수
def convert_text(text):
    text = text.replace("L", "리터")
    text = text.replace("kg.m", "킬로그램퍼미터")
    text = text.replace("cc", "씨씨")
    text = text.replace("kg", "킬로그램")
    text = text.replace("km", "킬로미터")
    text = text.replace("rpm", "알피엠")
    return text

# 음성 출력 함수
def text_to_speech(text, lang='ko'):
    # 특수문자를 제거하고 텍스트를 음성으로 변환
    cleaned_text = text.replace('*', '').replace('**', '')
    cleaned_text = convert_text(cleaned_text)  # 텍스트 변환 적용
    tts = gTTS(text=cleaned_text, lang=lang)
    tts.save("output.mp3")
    playsound("output.mp3")

## 10. 음성 관련 모듈
#### 10-1. 음성 인식 및 음원 초기화
이 코드는 두 개의 음원 파일 경로를 지정합니다. `ping_sound_path`는 "ping" 소리 파일의 경로이고, `ding_sound_path`는 "ding" 소리 파일의 경로입니다.

#### 10-2. 음원 재생 함수
이 함수는 주어진 경로의 음원 파일을 재생합니다.

1. `playsound(sound_path)`: `playsound` 라이브러리를 사용하여 `sound_path`에 지정된 경로의 음원 파일을 재생합니다.

이 함수는 입력받은 음원 파일 경로를 사용하여 해당 음원을 재생하는 역할을 합니다.

In [11]:
# 음성 인식 및 음원 초기화
ping_sound_path = "./sounds/ping-82822.mp3"
ding_sound_path = "./sounds/ding-36029.mp3"

# 음원 재생 함수
def play_sound(sound_path):
    playsound(sound_path)

## 11. 응답문 관련 처리
#### 11-1. 응답 처리 함수
`process_llm_response` 함수는 LLM(대형 언어 모델)의 응답을 처리합니다.

1. **응답이 문자열일 경우**:
   - `llm_response.replace('*', '').replace('**', '')`: 응답에서 '*'와 '**'를 제거합니다.
   - `cleaned_response = convert_text(cleaned_response)`: `convert_text` 함수를 호출하여 텍스트 내의 특정 단위를 한글 표현으로 변환합니다.
   - `print(cleaned_response)`: 변환된 응답을 출력합니다.

2. **응답이 문자열이 아닐 경우**:
   - `cleaned_response = llm_response.get("result", "").replace('*', '').replace('**', '')`: 응답 딕셔너리에서 "result" 키의 값을 가져와 '*'와 '**'를 제거합니다.
   - `cleaned_response = convert_text(cleaned_response)`: `convert_text` 함수를 호출하여 텍스트 내의 특정 단위를 한글 표현으로 변환합니다.
   - `print(cleaned_response)`: 변환된 응답을 출력합니다.
   - `print('\n\nSources:')`: 소스 문서 목록의 제목을 출력합니다.
   - `for source in llm_response.get("source_documents", [])`: 소스 문서들을 순회하며 각각의 출처를 출력합니다.
     - `print(source.metadata.get('source', 'Unknown'))`: 소스 문서의 메타데이터에서 'source'를 가져와 출력하고, 없으면 'Unknown'을 출력합니다.

3. **결과 반환**:
   - `return cleaned_response`: 변환된 응답을 반환합니다.

#### 11-2. 쿼리 및 요약 함수
`query_and_summarize` 함수는 사용자의 쿼리를 처리하고 요약된 응답을 제공합니다.

1. **입력 생성**:
   - `inputs = {"user_input": query}`: 사용자의 쿼리를 입력으로 설정합니다.

2. **LLM 호출**:
   - `llm_response = chain.invoke(inputs)`: 체인을 호출하여 쿼리에 대한 응답을 받습니다.

3. **응답 처리**:
   - `cleaned_response = process_llm_response(llm_response)`: `process_llm_response` 함수를 호출하여 응답을 처리합니다.

4. **쿼리와 응답 로깅**:
   - `log_query_and_response(query, cleaned_response)`: `log_query_and_response` 함수를 호출하여 쿼리와 응답을 로그에 기록합니다.

5. **텍스트를 음성으로 변환**:
   - `text_to_speech(cleaned_response)`: `text_to_speech` 함수를 호출하여 응답을 음성으로 변환합니다.

6. **결과 반환**:
   - `return cleaned_response`: 처리된 응답을 반환합니다.

In [12]:
# 응답 처리 함수
def process_llm_response(llm_response):
    if isinstance(llm_response, str):
        cleaned_response = llm_response.replace('*', '').replace('**', '')
        cleaned_response = convert_text(cleaned_response)  # 텍스트 변환 적용
        print(cleaned_response)
    else:
        cleaned_response = llm_response.get("result", "").replace('*', '').replace('**', '')
        cleaned_response = convert_text(cleaned_response)  # 텍스트 변환 적용
        print(cleaned_response)
        print('\n\nSources:')
        for source in llm_response.get("source_documents", []):
            print(source.metadata.get('source', 'Unknown'))
    return cleaned_response

def query_and_summarize(query):
    inputs = {"user_input": query}
    llm_response = chain.invoke(inputs)
    cleaned_response = process_llm_response(llm_response)
    log_query_and_response(query, cleaned_response)
    text_to_speech(cleaned_response)
    return cleaned_response

## 12. 음성 인식기 설정 및 취소 기능 추가

#### 12-1. 음성 인식기 초기화
`sr.Recognizer` 객체를 생성하여 음성 인식기를 초기화합니다. `is_listening` 변수는 음성 인식이 실행 중인지 여부를 추적하기 위해 사용됩니다.

#### 12-2. 음성 인식 및 처리 함수
이 함수는 음성을 인식하고 처리합니다.

1. **이미 음성 인식이 실행 중인 경우**:
   - `if is_listening`: 음성 인식이 이미 실행 중이라면 사용자에게 알림 메시지를 표시하고 함수를 종료합니다.

2. **음성 인식 시작**:
   - `is_listening = True`: 음성 인식이 시작되었음을 표시합니다.
   - `with sr.Microphone() as source`: 마이크로부터 음성을 입력받습니다.

3. **주변 소음 조정**:
   - `recognizer.adjust_for_ambient_noise(source, duration=1)`: 주변 소음 수준을 조정합니다.
   - `print("소음 조정 완료")`: 소음 조정 완료 메시지를 출력합니다.

4. **음성 입력 안내**:
   - `print("말씀하세요...")`: 사용자에게 음성 입력을 요청합니다.
   - `text_to_speech("말씀하세요")`: 음성으로 "말씀하세요"를 출력합니다.
   - `play_sound(ping_sound_path)`: "ping" 사운드를 재생합니다.

5. **음성 인식**:
   - `audio_data = recognizer.listen(source, timeout=5, phrase_time_limit=5)`: 음성을 입력받습니다. 시간 제한은 5초입니다.
   - `print("음성 인식 중...")`: 음성 인식 진행 메시지를 출력합니다.
   - `text = recognizer.recognize_google(audio_data, language='ko-KR')`: Google 음성 인식 API를 사용하여 한국어로 음성을 텍스트로 변환합니다.
   - `print("인식된 텍스트: " + text)`: 인식된 텍스트를 출력합니다.
   - `play_sound(ding_sound_path)`: "ding" 사운드를 재생합니다.
   - `query_and_summarize(text)`: 인식된 텍스트를 쿼리로 처리하고 요약합니다.

6. **오류 처리**:
   - `except sr.UnknownValueError`: 음성을 인식할 수 없는 경우 오류 메시지를 표시합니다.
   - `except sr.RequestError as e`: 음성 인식 서비스에 문제가 있는 경우 오류 메시지를 표시합니다.
   - `except sr.WaitTimeoutError`: 음성 인식 시간이 초과된 경우 오류 메시지를 표시합니다.

7. **음성 인식 종료**:
   - `finally: is_listening = False`: 음성 인식이 종료되었음을 표시합니다.

In [13]:
# 음성 인식기 설정 및 취소 기능 추가
recognizer = sr.Recognizer()
is_listening = False

def listen_and_process():
    global is_listening
    if is_listening:
        messagebox.showinfo("Info", "이미 음성 인식이 실행 중입니다.")
        return

    is_listening = True
    try:
        with sr.Microphone() as source:
            print("주변 소음 수준 조정 중...")
            recognizer.adjust_for_ambient_noise(source, duration=1)
            print("소음 조정 완료")

            print("말씀하세요...")
            text_to_speech("말씀하세요")
            play_sound(ping_sound_path)  # ping 사운드 재생
            audio_data = recognizer.listen(source, timeout=5, phrase_time_limit=5)
            print("음성 인식 중...")
            text = recognizer.recognize_google(audio_data, language='ko-KR')
            print("인식된 텍스트: " + text)
            play_sound(ding_sound_path)  # ding 사운드 재생
            query_and_summarize(text)  # query_and_summarize 함수 호출
    except sr.UnknownValueError:
        messagebox.showerror("Error", "음성을 인식할 수 없습니다")
    except sr.RequestError as e:
        messagebox.showerror("Error", f"음성 인식 서비스에 문제가 있습니다: {e}")
    except sr.WaitTimeoutError:
        messagebox.showerror("Error", "음성 인식 시간이 초과되었습니다")
    finally:
        is_listening = False

## 13. 음성 인식 중지 함수
1. **is_listening 상태 확인**:
   - `global is_listening`: 전역 변수 `is_listening`을 사용합니다.
   - `if is_listening`: 음성 인식이 실행 중인지 확인합니다.

2. **음성 인식 실행 중일 경우**:
   - `is_listening = False`: 음성 인식을 중지합니다. 실제로 `recognizer.listen()`을 중지하는 기능은 제공되지 않으므로, 단순히 상태 변수만 업데이트합니다.
   - `messagebox.showinfo("Info", "음성 인식이 취소되었습니다.")`: 사용자에게 음성 인식이 취소되었음을 알리는 메시지를 표시합니다.

3. **음성 인식이 실행 중이 아닐 경우**:
   - `messagebox.showinfo("Info", "현재 실행 중인 음성 인식이 없습니다.")`: 사용자에게 현재 실행 중인 음성 인식이 없음을 알리는 메시지를 표시합니다.

이 함수는 음성 인식이 실행 중인 경우 이를 중지하고 사용자에게 알림을 주며, 실행 중이 아닌 경우에도 사용자에게 현재 실행 중인 음성 인식이 없음을 알립니다.

In [14]:
def stop_listening():
    global is_listening
    if is_listening:
        is_listening = False
        messagebox.showinfo("Info", "음성 인식이 취소되었습니다.")
    else:
        messagebox.showinfo("Info", "현재 실행 중인 음성 인식이 없습니다.")

## 14. GUI 설정

#### 14-1. Tkinter 루트 윈도우 생성
`Tk()`를 사용하여 Tkinter 루트 윈도우를 생성하고, `title()` 메서드를 사용하여 창의 제목을 "음성 인식 차량 정보 시스템"으로 설정합니다.

#### 14-2. 프레임 생성 및 배치
`Frame` 위젯을 생성하여 루트 윈도우에 추가하고, `pack()` 메서드를 사용하여 프레임을 화면에 배치합니다. `pady=20` 옵션을 사용하여 프레임 위와 아래에 20픽셀의 패딩을 추가합니다.

#### 14-3. 시작 버튼 생성 및 배치
`Button` 위젯을 생성하여 프레임에 추가합니다. 버튼의 텍스트는 "음성 인식 시작"이며, 버튼이 클릭되면 `listen_and_process` 함수가 호출됩니다. `pack()` 메서드를 사용하여 버튼을 왼쪽(`side=tk.LEFT`)에 배치하고, 좌우에 10픽셀의 패딩을 추가합니다(`padx=10`).

#### 14-4. 종료 버튼 생성 및 배치
`Button` 위젯을 생성하여 프레임에 추가합니다. 버튼의 텍스트는 "종료"이며, 버튼이 클릭되면 애플리케이션이 종료됩니다(`root.quit`). `pack()` 메서드를 사용하여 버튼을 왼쪽(`side=tk.LEFT`)에 배치하고, 좌우에 10픽셀의 패딩을 추가합니다(`padx=10`).

#### 14-5. 메인 이벤트 루프 시작
`mainloop()` 메서드를 호출하여 Tkinter 메인 이벤트 루프를 시작합니다. 이 루프가 실행되는 동안 GUI는 사용자 입력을 기다리고 처리합니다.

이 GUI 설정 코드는 간단한 사용자 인터페이스를 구성하여 음성 인식을 시작하고 애플리케이션을 종료할 수 있는 버튼을 제공합니다.

In [ ]:
# GUI 설정
root = tk.Tk()
root.title("음성 인식 차량 정보 시스템")

frame = tk.Frame(root)
frame.pack(pady=20)

start_button = tk.Button(frame, text="음성 인식 시작", command=listen_and_process)
start_button.pack(side=tk.LEFT, padx=10)

close_button = tk.Button(frame, text="종료", command=root.quit)
close_button.pack(side=tk.LEFT, padx=10)

root.mainloop()

주변 소음 수준 조정 중...
소음 조정 완료
말씀하세요...
음성 인식 중...
인식된 텍스트: k3 엔진오일 교환 주기 알려 줘
기아 K3의 엔진오일 교환 주기는 일반적으로 10,000킬로미터 또는 1년 중 먼저 도래하는 시점입니다. 하지만 운전 습관이나 주행 환경에 따라 다를 수 있으니, 차량 매뉴얼을 참고하거나 정비소에서 권장하는 주기를 따르는 것이 좋습니다.


Sources:
